In [1]:
## Text Mining Project
# imports
import pandas as pd
import json
import re
import lyricsgenius
import time

In [6]:
# Genius token - https://genius.com/api-clients
# Library - https://github.com/johnwmillr/LyricsGenius
token = "KbIhkIUKtt6IqQnUWh7KwZXWU8d-FduTSQfCnJWHQh97BVs07PisfSWIYs0uAwX6"
genius = lyricsgenius.Genius(token)

In [8]:
# Download the lyrics
all_artists = ['187 Strassenbande', 'AK Ausserkontrolle', 'Alligatoah', 'Apache 207', 'Azad', 'Azet', 'badmómzjay', 'Bausa', 'Beginner', 'Bonez MC', 'Bushido', 'Capital Bra', 'Casper', 'Celo & Abdi', 'Chakuza', 'Cro', 'Culcha Candela', 'CÉLINE', 'Dardan', 'Deichkind', 'Die Atzen', 'Die Fantastischen Vier', 'Eko Fresh', 'Eno', 'Farid Bang', 'Fero47', 'Fettes Brot', 'Fler', 'Freundeskreis', 'Genetikk', 'Gzuz', 'Haftbefehl', 'Jamule', 'Jan Delay', 'Joker Bra', 'Juju', 'K.I.Z', 'KC Rebell', 'Kay One', 'Kollegah', 'Kontra K', 'Kool Savas', 'Kraftklub', 'Loredana', 'Luciano', 'MERO', 'Marteria', 'Max Herre', 'MoTrip', 'Money Boy', 'Moses Pelham', 'Nimo', 'Olexesh', 'Peter Fox', 'Prinz Pi', 'RAF Camora', 'RIN', 'Rödelheim Hartreim Projekt', 'SSIO', 'SXTN', 'Sabrina Setlur', 'Samra', 'Samy Deluxe', 'Seeed', 'Shindy', 'Sido', 'Silla', 'Summer Cem', 'Tic Tac Toe', 'Trailerpark', 'Trettmann', 'Ufo361', 'Veysel', 'XATAR', 'Xavas', 'Yung Hurn', 'Zuna']

# for artist_name in all_artists:
#     try:
#         artist = genius.search_artist(artist_name, sort="title")
#         artist.save_lyrics()
#     except:
#         time.sleep(300)
#         artist = genius.search_artist(artist_name, sort="title")
#         artist.save_lyrics()

Searching for songs by XATAR...

Song 1: "2 Baba Parts"
Song 2: "415"
Song 3: "415 (Knastversion)"
Timeout raised and caught:
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
Searching for songs by XATAR...

Song 1: "2 Baba Parts"
Song 2: "415"
Song 3: "415 (Knastversion)"
Song 4: "AGB 2010 (10 Regeln)"
Song 5: "Alles oder Nix"
Song 6: "AON 2018"
"AON 2018 (Instrumental)" is not valid. Skipping.
Song 7: "AON Crü"
Song 8: "AON Spezial"
Song 9: "Attrappen"
Song 10: "Baba aller Babas"
"Baba aller Babas - Alle Infos, Interviews und Videos" is not valid. Skipping.
Song 11: "Baba aller Babas im Land"
Song 12: "Balla"
"Balla (Instrumental)" is not valid. Skipping.
Song 13: "Balla (Remix)*"
Song 14: "Befeindete Freunde"
Song 15: "Beifall"
Song 16: "Blaues Häkchen"
Song 17: "B.O.X."
Song 18: "Dein Herz klopft"
"Der Holland Job - Alle Infos, Interviews und Videos" is not valid. Skipping.
Song 19: "Die Welt gehört dir"
Song 20: "Dresscode"
"Ebbe & Flut (Instr

Song 31: "Für die Million"
Song 32: "Ghassan*"
Song 33: "Ghetto"
"Ghetto (Instrumental)" is not valid. Skipping.
Song 34: "Grüße an die Cops"
Song 35: "Guck Mama"
Song 36: "Guck Mama 2*"
"Guck Mama (Instrumental)" is not valid. Skipping.
Song 37: "Habibte*"
Song 38: "Hinter der Rayban, Pt. 2"
Song 39: "Hol mir dein Cousin"
Song 40: "Homie"
Song 41: "Hurensohn"
Song 42: "Intro"
Song 43: "Kartell"
Song 44: "Kerosin (Snippet)"
Song 45: "KMN"
"KMN (Instrumental)" is not valid. Skipping.
Song 46: "Knie vor Gott"
Song 47: "Kriminell"
Song 48: "Lambo*"
Song 49: "La Vida loca (Snippet)"
Song 50: "Leyla*"
Song 51: "Lhamdela"
"Lhamdela (Instrumental)" is not valid. Skipping.
Song 52: "Marihuana (Snippet)"
Song 53: "Mele7"
"Mele7 (Instrumental)" is not valid. Skipping.
Song 54: "Million (Snippet)"
Song 55: "Nike Pullover (Snippet)"
Song 56: "Nshallah"
Song 57: "Nummer 1"
"Nummer 1 (Instrumental)" is not valid. Skipping.
Song 58: "Okay (Snippet)"
Song 59: "Original"
"Original (Instrumental)" is no

In [87]:
# Store lyrics and relevant metadata in dataframe
all_songs = pd.DataFrame(columns = ['Title', 'Artist', 'Release', 'Lyrics'])

for artist_name in all_artists:
    print(artist_name)
    # Extract data from the json files
    with open(f'Lyrics_{artist_name.replace(" ", "").replace("&", "")}.json') as json_file:
        data = json.load(json_file)
        for song in data['songs']:
            title = song['title']
            release = song['release_date']
            lyrics_raw = song['lyrics']
            # Exclude songs without lyrics or release date, songs that contain brackets () and commentaries
            if lyrics_raw != None and release != None and not re.search(' \(.*?\)', title) and 'Commentary' not in title:
                # Remove [] brackets from the lyrics, as they only contain meta info e.g. [Hook], [Intro]
                lyrics = re.sub("[\[].*?[\]]", "", lyrics_raw)
                new_song = {'Title': title, 'Artist': artist_name, 'Release': release, 'Lyrics': lyrics}
                all_songs = all_songs.append(new_song, ignore_index=True)

187 Strassenbande
AK Ausserkontrolle
Alligatoah
Apache 207
Azad
Azet
badmómzjay
Bausa
Beginner
Bonez MC
Bushido
Capital Bra
Casper
Celo & Abdi
Chakuza
Cro
Culcha Candela
CÉLINE
Dardan
Deichkind
Die Atzen
Die Fantastischen Vier
Eko Fresh
Eno
Farid Bang
Fero47
Fettes Brot
Fler
Freundeskreis
Genetikk
Gzuz
Haftbefehl
Jamule
Jan Delay
Joker Bra
Juju
K.I.Z
KC Rebell
Kay One
Kollegah
Kontra K
Kool Savas
Kraftklub
Loredana
Luciano
MERO
Marteria
Max Herre
MoTrip
Money Boy
Moses Pelham
Nimo
Olexesh
Peter Fox
Prinz Pi
RAF Camora
RIN
Rödelheim Hartreim Projekt
SSIO
SXTN
Sabrina Setlur
Samra
Samy Deluxe
Seeed
Shindy
Sido
Silla
Summer Cem
Tic Tac Toe
Trailerpark
Trettmann
Ufo361
Veysel
XATAR
Xavas
Yung Hurn
Zuna


In [95]:
#print(test_frame[test_frame['Release'].isnull()])
#print(test_frame[test_frame['Title'].str.contains(' \(.*?\)')])
#print(all_songs[all_songs['Artist']=='SXTN'])
#print(all_songs)